<center><img src="https://github.com/DACSS-Spatial/data_forSpatial/raw/main/logo.png" width="700"></center>







<a target="_blank" href="https://colab.research.google.com/github/DACSS-Spatial/spatial_autoCorr/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

-------

# Neighboorhood and Spatial Correlation.



## Getting ready


### Libraries needed

Let's verify:

In [ ]:
!pip show pysal pandas geopandas

In [ ]:
## needed in Colab
# !pip install pysal

### Data to use

Let me get two maps:

1. Here we have the USA map, at **state level**,  directly from census.gov, which has a good quality.

In [ ]:
import geopandas as gpd

url = "https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_state_500k.zip"
us_states = gpd.read_file(url)

# checking data type,  crs, and confirming crs is projected
us_states.info(),us_states.crs.to_epsg(),us_states.crs.is_projected

Notice this map has basic information per state:

In [ ]:
us_states.head()

Also, notice the current **unprojected** crs will plot this:

In [ ]:
us_states.plot()

Let's reproject the crs:

In [ ]:
us_states=us_states.to_crs(5070)
us_states.plot()

Let's use the state name **as index**, that would help an easier identification of the places when we see most outputs (otherwise we will see just numerical indexes) :

In [ ]:
us_states.set_index('NAME', inplace=True)
us_states.head()

Let's keep some states, which will help visuallly in most examples (mainly Part I,  and II):

In [ ]:
someStates=['Utah','Colorado','Arizona','New Mexico', 'Florida','Georgia','Alabama']
sub_us=us_states[us_states.index.isin(someStates)]
sub_us

2. A map of Peru, at the **'distrito'** level (similar to municipality in the USA - not exactly the same). The map comes from an unoffical [website](https://www.geogpsperu.com/p/descargas.html). Some columns with **social data** have been added. This will be used at the end of this material.

In [ ]:
peruDataLink="https://github.com/DACSS-Spatial/data_forSpatial/raw/refs/heads/main/PERU/PeruMaps.gpkg"
peru_distritos=gpd.read_file(peruDataLink,layer='distritos')

# some basic info
peru_distritos.info(),peru_distritos.crs.to_epsg(),peru_distritos.crs.is_projected

Let's reproject and plot:

In [ ]:
peru_distritos=peru_distritos.to_crs(32718)
peru_distritos.plot()

Besides the spatial units (DEPARTAMEN, PROVINCIA, DISTRITO, and Ubigeo - "Ubigeo" is a code ), you have:
 - **Poblacion**: Population (2017)
 - **Superficie**: Area               
 - **IDH2019**: Human Development Index for DISTRITO (2019)                   
 - **Educ_sec_comp2019_pct**: Share of Population that finished High-School (2019)     
 - **NBI2017_pct**: Share of Population with poverty at the household level aggregated by DISTRITO. This index ("Unsatisfied Basic Needs") uses observable living conditions rather than income alone (2017).
 - **Viv_sin_serv_hig2017_pct**: Share of housing units that have no sanitation infrastructure aggregated by  DISTRITO (2017)

The last two sections (III and IV) will need these **social data**.


Notice we should not use the 'distrito' name as index, because several of them are repeated:

In [ ]:
peru_distritos[peru_distritos['DISTRITO'].duplicated()]

Let's use 'Ubigeo', although it is not the best solution

In [ ]:
# of course
peru_distritos[peru_distritos['Ubigeo'].duplicated()]

In [ ]:
#then
peru_distritos.set_index('Ubigeo', inplace=True)

**NOTICE** both projections used above are equal-area ones, which are standard for distance computations in meters.

## I. Who is my neighbor?

In spatial analysis, the intuitive concept of a “neighbor” can be operationalized in multiple ways.

So far, we have identified neighbors using geometric operations such as buffering, spatial joins, and overlays. 

In this session, let’s remember the distance matrix:


In [ ]:
sub_us.geometry.apply\
(lambda state: sub_us.distance(state)/1000)

You can see roughly the distances here:

In [ ]:
# Get the bounding box coordinates of sub_us
minx, miny, maxx, maxy = sub_us.total_bounds

base = us_states.plot(facecolor='white', edgecolor='lightgrey')
sub_us.plot(ax=base)
# Set the x and y limits of the plot to the bounding box
base.set_xlim(minx, maxx)
base.set_ylim(miny, maxy)

From this matrix and plot, you’ll notice that neighboring features have a distance of zero—this occurs when two polygons share a boundary (i.e., they are contiguous). This observation helps illustrate the different approaches to defining neighbors:

1. Binary Relationships:

* Contiguity: Two polygons are considered neighbors if they share any portion of their boundary—whether at a point, along a line segment, or more extensively. In the distance matrix, such pairs show a distance of zero, reflecting direct spatial adjacency.

    * Queen contiguity
    * Rook contiguity 

* Proximity: This approches do not require contiguity.
    - Nearest Neighbors
    - Within influence zone

2. Continuous Relationships

* Contiguity: The relationship is graded by  the length of the shared boundary between contiguous neighbors.

* Proximity:
    - Nearest neighbors.
    - Within influence zone
 

Using matrices for this relationships—rather than raw geometries—is essential for the mathematical abstraction/representation and numerical computation required in upcoming spatial analytics techniques.

We will introduce some of PySAL (libpysal), which is designed to handle spatial relationship matrices and integrates seamlessly with GeoPandas. PySAL uses graph-based representations of these matrice, which  is not only more memory-efficient but also speeds up computation and simplifies visualization—especially for large or sparse spatial datasets.  A key concept in these graphs is the distinction between:

- Focal unit: the spatial feature (e.g., a state, county, or census tract) for which we are identifying neighbors.
- Neighbor(s): the other spatial units that are considered related to the focal unit based on a chosen criterion (e.g., contiguity, distance, or shared border length).

Let’s become familiar with the ```graph``` module in PySAL/libpysal, which provides a modern and flexible framework for constructing and working with spatial neighbor graphs.


In [ ]:
from libpysal.graph import Graph

# also for reproducibility!
from numpy.random import seed
seed(42)

### I.1 Binary Contiguity

Take a look at the **queen** and **rook** relationship:

<center><img src="https://github.com/CienciaDeDatosEspacial/spatial_autoCorr/raw/main/rookQueen.png" width="700"></center>

From the image above:
- Your **rook** neighbor is whichever  shares a border with you (a borderline of at least two points). It is also known as the _Von Neumann_ neighbor.

- Your **queen** neighbor is whichever  shares a border or a corner with you (at least one point).It is also known as the _Moore_ neighbor.

Let's see how to get each set of neighbors

#### I.1.A. Rook

* The key idea: A focal polygon considers another polygon a neighbor if they share a common edge—that is, two or more connected points forming a line segment of positive length.
* The input: A GeoDataFrame (GDF) containing polygon geometries (e.g., states, counties, census tracts). Each row represents a spatial unit that can serve as a focal observation.
* The process: For each polygon (focal unit), the algorithm checks all other polygons to determine whether their boundaries intersect along a line segment (not just at a point).
* The output: A graph built from a binary adjacency matrix where each node represents a polygon (focal unit). An edge exists between two nodes only if they share a boundary segment. The corresponding adjacency matrix is binary. Isolates are possible.

Let's use **sub_us** GDF:

In [ ]:
rook=Graph.build_contiguity(sub_us,rook=True) # rook by default

Now let's check the ouput:

**a. adjacency**

In [ ]:
rook.adjacency

The previous results shows only the neighbors of the focals, to recreate a wide format:

In [ ]:
import pandas as pd
pd.DataFrame(rook.adjacency).unstack()

We generally fill those missing values (not a neighbor) with zero.

In [ ]:
rook_Matrix=pd.DataFrame(rook.adjacency).unstack().fillna(0)
rook_Matrix

SInce we are going to use several times, we may create a custom procedure:

In [ ]:
#instead of
# pd.DataFrame(rook.adjacency).unstack().fillna(0)
toWideMatrix=lambda g:pd.DataFrame(g.adjacency).unstack().fillna(0)

Notice we created this wide matrix for pedagogical purposes. It is a bad idea if you have many columns and rows.

**b. adjacency graph**

As we have a `GRAPH`, we can identify these neighborhood relationships via edges:

In [ ]:
##this will be used several times:
general_arguments=dict(gdf=sub_us,node_kws=dict(color='red'), edge_kws=dict(alpha=0.4,color='blue'),zoom_start = 6)
rook.explore(**general_arguments)

#### I.1.B. Queen

* The key idea: A focal polygon considers another polygon a neighbor if they share a common edge or a vertex (at least one point).
* The input: A GeoDataFrame (GDF) containing polygon geometries (e.g., states, counties, census tracts). Each row represents a spatial unit that can serve as a focal observation.
* The process: For each polygon (focal unit), the algorithm checks all other polygons to determine whether their boundaries intersect at any point or along a line segment.
* The output: A graph built from a binary adjacency matrix where each node represents a polygon (focal unit). An edge exists between two nodes only if they share a point or boundary segment. The corresponding adjacency matrix is binary. Isolates are possible.

Let's see what we get:

**a. adjacency matrix**

In [ ]:
queen=Graph.build_contiguity(sub_us,rook=False)

# applying our procedure:

queen_Matrix=toWideMatrix(queen)
queen_Matrix

**b. adjacency plot**

In [ ]:
queen.explore(**general_arguments)


### I.2 Binary Proximity


#### I.2.A. Nearest Neighbors

* The key idea: A focal polygon considers another polygon a neighbor if that second polygon is among the K closest polygons to the focal polygon, where “closeness” is measured by the distance between their geometric centroids (or any other user-supplied distance metric).  
* The input: A GeoDataFrame (GDF) containing polygon geometries. Each row represents a spatial unit that can serve as a focal observation.  
* The process: For each polygon (focal unit), the algorithm (1) computes the distance between its centroid and every other centroid, (2) ranks these distances, and (3) flags the K smallest distances as neighbors. Ties can be broken by random selection, ID order, or by including all tied candidates.  
* The output: A graph built from a binary adjacency matrix where each node represents a polygon (focal unit). An edge exists between two nodes whenever one polygon is among the K nearest neighbors of the other. The corresponding adjacency matrix is binary and, by construction, typically asymmetric unless a symmetric constraint is explicitly enforced. No isolates in output.

If assume K is 3:

**a. adjacency matrix**

In [ ]:
knn3 = Graph.build_knn(sub_us.representative_point(), # GDF
                                 k=3) # desired k

knn3_Matrix=toWideMatrix(knn3)
knn3_Matrix

**b. adjacency plot**

In [ ]:
knn3.explore(**general_arguments)

#### I.2.B Within zone of influence


* The key idea: A focal polygon considers another polygon a neighbor if the distance between them (usually centroid-to-centroid , or a pair of representative point) lies within a user-defined band.
* The input: A GeoDataFrame (GDF) containing polygon geometries for which a representative point is supplied. Each row represents a spatial unit that can serve as a focal observation, plus (optionally) an attribute column to use as a weight or ID.  
* The process: For each polygon (focal unit) the algorithm computes the distance to every other polygon and flags those whose distance falls inside the specified band.
* The output: A binary spatial graph (or adjacency structure) where each node represents a polygon. An edge exists between two nodes only if their mutual distance sits inside the chosen band. The corresponding adjacency matrix is binary and symmetric by construction. Isolates are possible.

Let's assume a 750 km distance band:

In [ ]:
band750k_Bi=Graph.build_distance_band(sub_us.representative_point(), threshold=750000)

band750k_Bi_Matrix=toWideMatrix(band750k_Bi)
band750k_Bi_Matrix

In [ ]:
band750k_Bi.explore(**general_arguments)

### I.3 Continuos Proximity

#### I.3.A. Within zone of influence

* The key idea: A focal polygon considers another polygon a neighbor if the distance between them (usually centroid-to-centroid , or a pair of representative point) lies within a user-defined band.
* The input: A GeoDataFrame (GDF) containing polygon geometries for which a representative point is supplied. Each row represents a spatial unit that can serve as a focal observation, plus (optionally) an attribute column to use as a weight or ID.  
* The process: For each polygon (focal unit) the algorithm computes the distance to every other polygon and flags those whose distance falls inside the specified band.
* **The output**: a graph where each node has edges to all nodes ('neighbors') whose representative points lie within the radius. Edges are represented by a continuous adjacency matrix, the default values represent the inverse distance between the nodes in the edges. Isolates are possible

Here it is:

In [ ]:
band2Mk_C=Graph.build_distance_band(sub_us.representative_point(), threshold=2000000,binary=False)

band2Mk_C_Matrix=toWideMatrix(band2Mk_C)
band2Mk_C_Matrix


The plot:

In [ ]:
band2Mk_C.explore(**general_arguments)

Above, you do not see the continuous information present in the edges, for that, you may plot one of the states using **focal=**:

In [ ]:
# pay attention to one state:
focal_args=dict(gdf=sub_us,focal='Arizona',edge_kws=dict(column="weight",style_kwds=dict(weight=6)),zoom_start=6)
band2Mk_C.explore(**focal_args)


#### I.3.B. Kernel K-Nearest Neighbors (KNN)

* The key idea: Each polygon keeps only other  K closest polygons, but instead of a blunt 0 / 1 the tie is a gentle “friendliness” score that shrinks with distance.  
* The input: A GeoDataFrame of polygons + their representative points.  
* The process:  
  – Find the K nearest points.  
  – Turn distance into a Gaussian bell value (big when close, small when far).  

* The output: A weighted graph where every node has exactly K neighbours carrying continuous “soft” weights; no isolates, no huge clumps, just K tidy friendship levels per row.

Let's use K = 3 again:

In [ ]:
kernel3 = Graph.build_kernel(sub_us.representative_point(), k=3)

kernel3_Matrix=toWideMatrix(kernel3)
kernel3_Matrix

Notice we have bigger values that the continuous band-based result. Let's see the basic plot:

In [ ]:

kernel3.explore(**general_arguments)

Focusing on one state:

In [ ]:

kernel3.explore(**focal_args)

### I.4 Continuous Contiguity


* **The key idea**: A focal polygon treats another polygon as a neighbour only if the two share a common border.  
* **The input**: A GeoDataFrame of polygons; no extra points needed.  
* **The process**:  
  – Compute the exact length of every shared boundary segment.  

  
* **The output**: A weighted graph encoded in a sparse weights matrix; each non-zero entry is the **metres (or map-units) of shared border**, giving large, jagged polygons more influence over their neighbours than small, compact ones. Isolates are possible.

In [ ]:
perimeter = Graph.build_contiguity(sub_us, by_perimeter=True) #rook by default

perimeter_Matrix=toWideMatrix(perimeter)
perimeter_Matrix


The basic plot:

In [ ]:

perimeter.explore(**general_arguments)

Focusing on one state:

In [ ]:

perimeter.explore(**focal_args)

In summary:

| Technique | Pros | Cons | Typical use-case example |
|-----------|------|------|--------------------------|
| **Rook / Queen contiguity** | Simple, law- or admin-based, no distance parameter | Islands get 0 neighbours; ignores “near but not touching” units | State-to-state policy diffusion, local tax spill-overs |
| **K-NN (binary)** | Guarantees every unit has same #neighbours; no islands | Can link far-away units in sparse areas; ignores true borders | Comparative politics with same-size legislatures, small-N samples |
| **Distance band (binary)** | Hard cap on geographic reach; easy to interpret radius | Sparse → islands; dense → huge cliques; radius must be tuned | Housing-price spill-overs within 30 km commuting range |
| **Continuous distance-band** | Smooth weights, still respects max range | Same radius-tuning and island issues as binary band | Environmental exposure declining with distance, gravity models |
| **Kernel K-NN** | Fixed neighbour count, adaptive bandwidth, no islands | Far neighbours possible in deserts/oceans; parameter k to pick | Identifying neighbors that contribute to a smoothly decaying environmental hazard level.|
| **Perimeter-weighted contiguity** | Uses real boundary, good for irregular shapes | Needs clean topology; computation heavier | Ecological edge effects, river-basin pollution, crop pest spread |

## II. Neighborhoods as weights

Our matrices tell us which are neighbors, and some give us additional information related to the 'farness' of the identified neighbor. Let me compute the marginal values by row (sum):

In [ ]:
allMx=[rook_Matrix.sum(axis=1),
       queen_Matrix.sum(axis=1),
       knn3_Matrix.sum(axis=1),
       band750k_Bi_Matrix.sum(axis=1),
       band2Mk_C_Matrix.sum(axis=1),
       kernel3_Matrix.sum(axis=1),
       perimeter_Matrix.sum(axis=1)]
pd.concat(allMx,axis=1)

We know that no-neighbors have value zero in the matrix. Then, non-zero (binary or non-binary) values carries some weight .
However, they are currently not normalized (they do not add to 1 by row), which may bias further procedures as raw values may 'distort' mathematical modelling.

For example, this binary matrix is non-normalized:

In [ ]:
queen_Matrix

But this binary matrix is normalized (by **r**ows):

In [ ]:
toWideMatrix(queen.transform("r"))

Another example, this binary matrix is non-normalized

In [ ]:
perimeter_Matrix

We know:

In [ ]:
pd.concat([perimeter_Matrix,perimeter_Matrix.sum(axis=1).rename("SUM")],axis=1)

Here it is normalized:

In [ ]:
toWideMatrix(perimeter.transform("r"))

Let's confirm:

In [ ]:
Mtrans = toWideMatrix(perimeter.transform("r"))

# CORRECTED CODE
pd.concat([Mtrans, Mtrans.sum(axis=1).rename("SUM")], axis=1)

These new row-standardized matrices will serve a greater purpose: the computing of **spatial lags**.

## III. How are my neighbors doing...?

The actual question should be related to a social variable, for example, poverty, mortality, education, etc.

For example, this is "how each distrito is doing" related to HS education (share of population that completed high school):

In [ ]:
peru_distritos.Educ_sec_comp2019_pct.describe()

See the choropleth (quantile bining):

In [ ]:
peru_distritos.plot(
    "Educ_sec_comp2019_pct",
    scheme="quantiles",
    cmap="Reds_r",
    legend=True,figsize=(12, 10))

The title was asking about the situation of neighbors (the **neighbors** of each **focal** in my data). In this case, each focal has an education indicator (value), then, the average of the neighbors of each focal is a new value (new column) which we call **spatial lag** of that variable.

We do not have yet Peru's  neigborhood graph for the adjacency matrix. Let's try to get it: 

In [ ]:
peru_perim=Graph.build_contiguity(peru_distritos,by_perimeter=True)

We are in trouble, the map quality is not good. This will always be a key consideration.

I have another map with data, based on an official [website](https://www.geoidep.gob.pe/servicios-idep/servicios-de-publicacion-de-objetos-wfs).

In [ ]:
peru_good=gpd.read_file(peruDataLink,layer='good_geom')

Let's retry:

In [ ]:
peru_perim=Graph.build_contiguity(peru_good,by_perimeter=True)

This time it worked!

Now, let's get the normalized matrix:

In [ ]:
peru_perim=peru_perim.transform("r")

Then,  the spatial lag of the variable "Educ_sec_comp2019_pct" can be obtained like this:

In [ ]:
ylag = peru_perim.lag(peru_good["Educ_sec_comp2019_pct"])

Let me add it to the GDF:

In [ ]:
peru_good=peru_good.assign(Educ_sec_comp2019_pct_lagged=ylag)
peru_good.head()

Let's plot HS share against its lag:

In [ ]:
peru_good.plot.scatter("Educ_sec_comp2019_pct","Educ_sec_comp2019_pct_lagged")

## IV. Spatial autocorrelation

### IV. 1 Global

A correlation plot as the previous one should let us know if a variable is correlated with values of the neighbors (the lag), and if that were the case,  proximity is interfering statistical analysis, as the variable values are not independent.

The most well known measure to confirm that is the Moran's I index of global spatial correlation. Let's see the value we get:


In [ ]:
import esda # from pysal

MoranGlobal_HS = esda.Moran(peru_good['Educ_sec_comp2019_pct'], peru_perim)

Once computed, you can retrieve its value and significance:

In [ ]:
MoranGlobal_HS.I,MoranGlobal_HS.p_sim

Moran’s I is interpreted like a Pearson r, but benchmarks are lower:

- |I| > 0.5 → strong, 0.2–0.5 → moderate, < 0.2 → weak.
- Sign indicates positive vs. negative spatial clustering.

Significance is assessed with a pseudo p-value from conditional randomisations of the data, because the theoretical null distribution is unknown for irregular spatial weights.

### IV. 2 Local

We can compute a Local Index of Spatial Association (LISA -local Moran) for each map polygon. That will help us find spatial clusters (spots) and spatial outliers:

- High-High (HH): values above average surrounded by values above average.These are also known as **hotSpot**s.
- Low-Low (LL): values below average surrounded by values below average. These are also known as **coldSpot**s.
- High-Low (HL): values above average surrounded by values below average.These are also known as **hotOutlier**s.
- Low-High (LH): values below average surrounded by values above average. These are also known as **coldOutlier**s.

It is also possible that no significant correlation is detected. Let's see those values:



a. Compute the LISAs:

In [ ]:
lisa = esda.Moran_Local(peru_good['Educ_sec_comp2019_pct'], peru_perim)


The previous warning may be due to the presence of isolates, produced by perimeter algorithm:

In [ ]:
peru_perim.isolates

These "islands" may create trouble. Should you keep using a _contiguity_ approach, you may consider fltering those out and recompute ``graph`` and ``lisa``:

In [ ]:
islandCodes = peru_perim.isolates

peru_clean=peru_good.drop(index=islandCodes).copy()

peru_perim_clean=Graph.build_contiguity(peru_clean,by_perimeter=True).transform("r")
#
lisa = esda.Moran_Local(peru_clean['Educ_sec_comp2019_pct'], peru_perim_clean)



b. Plot the results (**fast way**)

In [ ]:
lisa.plot(peru_clean,crit_value=0.05,figsize=(10,13),legend=True)

#### IV. 2. 1 Analysis Local Spatial Correlation quadrants

You can not get the quadrant labels from the previous plot directly into the original GDF, thus complicating further analysis and exporting of those results.

First let's recover the labels into a new column:

In [ ]:
# get the quadrants
peru_clean['HS_lisa'] = lisa.get_cluster_labels(crit_value=0.05)

In [ ]:
# see the count
peru_clean['HS_lisa'].value_counts()

Now, we can prepare a comparisson:

In [ ]:
TheStats=["mean", "min", "max","var"]


# stats by the lisa cluster
grouped_stats = peru_clean.groupby('HS_lisa')['Educ_sec_comp2019_pct'].agg(TheStats)

# stats for the entire, ungrouped column
global_stats = peru_clean['Educ_sec_comp2019_pct'].agg(TheStats)

# Convert 'global_stats' (Series) to a DataFrame with an appropriate index name
global_stats_df = pd.DataFrame(global_stats).T
global_stats_df.index = ['Global/Total'] # simple but key

# Combine both
pd.concat([grouped_stats, global_stats_df])



From the results above, you may have several concerns and keep asking more questions:

- High-High clusters are concentrated areas of high percent of graduation from HighSchool. Is that expected in those areas? What has caused that? Has this been the same for a long time?

- Low-Low clusters are concentrated areas of low percent of graduation from HighSchool. What would it take to make a change in these areas? No doubt some kind of intervention is needed.

- High-Low clusters detect districts that outperform their neighbors. Can we leverage on those to improve the area? Are they in danger of worsening?

- Low-High clusters detect districts that are much worse than their neighbors, who are actually doing pretty well. Can we improve this place considering the surroundings need less intervention or not at all?

#### IV. 2. 2 Customizing Local Spatial Correlation quadrants

At this stage, we can rename the previous column to customize cluster labels and colors:

In [ ]:
##knowing
peru_clean['HS_lisa'].unique()

Keep in mind that the outliers may deserve more attention, so they may be at the top or bottom of the levels. You can add a number to achieve this purpose:

In [ ]:
oldLabels=['Insignificant', 'Low-Low', 'High-High', 'High-Low', 'Low-High']

# 3 will be at the middle
newLabels = ['3 no_pattern', '4 coldSpot','2 hotSpot','1 hotOutlier' , '5 coldOutlier']

labels = dict(zip(oldLabels, newLabels))

labels

Just using this dictionary to rename the column:

In [ ]:

peru_clean.replace({'HS_lisa':labels},inplace=True)

## see the count
peru_clean['HS_lisa'].value_counts().sort_index()

Finally, used that order to assign diverging color-blind safe palette:

In [ ]:
##custom colors- respect the previous order

import matplotlib.pyplot as plt
myColMap = plt.get_cmap('PuOr', 5)


##plot the map

peru_clean.plot(column='HS_lisa',
                categorical=True,
                cmap=myColMap,
                linewidth=0.1,
                edgecolor='k',
                legend=True,
                legend_kwds={'bbox_to_anchor': (0.3, 0.3)},
                figsize=(12,12))